# Preguntas
- Cuánto pagan de expensas los departamentos en CABA según barrio?
  - La media de las expensas según barrio
  - Media del ratio entre alquiler y expensas
  - media de expensas por m2

In [123]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [124]:
palabras_identifican_no_vivienda = ["oficina", "torre", "local", "oficinas"]

In [125]:
df = pd.read_parquet("output/zonaprop.pkt")

In [126]:
conditions = [
    df['Price'].str.startswith('USD'),
    df['Price'].str.startswith('$'),
    df['Price'].str.startswith('Consultar')
]

values = ['USD', 'ARS', 'Consultar']

df['currency'] = np.select(conditions, values, default=np.nan)
df

,Price,Location,Address,Features,Description,Expensas,Link,currency
0,USD 22.460,"Puerto Madero, Capital Federal",NaN,"[ 471 m² , 471 m² , 456 m² , 456 m² , 6 am...",Alquiler de Departamento 6 ambientes en Puerto...,$ 280.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,USD
1,USD 48.000,"Palermo, Capital Federal",NaN,"[ 1500 m² , 1500 m² , 1500 m² , 1500 m² ]","Ubicado en Palermo, sobre Av. Del Libertador y...",None,https://www.zonaprop.com.ar/propiedades/clasif...,USD
2,USD 7.500,"Recoleta, Capital Federal",NaN,"[ 372 m² , 372 m² , 372 m² , 372 m² , 2 ba...","Linda oficina en Recoleta, piso alto, amplia y...",$ 51.500 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,USD
3,$ 260.000,"Flores, Capital Federal",NaN,"[ 53 m² , 53 m² , 53 m² , 53 m² , 2 amb. ,...",Dos ambientes dividido con Durlock. Oficinas y...,$ 32.760 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS
4,USD 6.720,"Monserrat, Capital Federal",NaN,"[ 408 m² , 408 m² , 8 baños , 8 baños , 3 ...",¡¡oficina comercial!! Todo el piso Piso recién...,$ 480.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,USD
...,...,...,...,...,...,...,...,...
18041,$ 700.000,"Recoleta, Capital Federal",NaN,"[ 75 m² , 75 m² , 70 m² , 70 m² , 3 amb. ,...","Excelente tres ambientes Recoleta, un departam...",$ 85.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS
18042,$ 150.000,"Liniers, Capital Federal",NaN,"[ 30 m² , 30 m² , 30 m² , 30 m² , 1 baño ,...",Xintel(fde-fde-930) Alquiler de Local en Linie...,$ 25.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS
18043,$ 500.000,"Villa Devoto, Capital Federal",NaN,"[ 70 m² , 70 m² , 70 m² , 70 m² , 1 baño ,...",Local alquiler Villa Devoto. Amplio local en a...,None,https://www.zonaprop.com.ar/propiedades/clasif...,ARS
18044,$ 250.000,"Villa Crespo, Capital Federal",NaN,"[ 70 m² , 70 m² , 70 m² , 70 m² , 1 baño ,...",Local comercial sobre av. Scalabrini ortiz N° ...,None,https://www.zonaprop.com.ar/propiedades/clasif...,ARS


In [127]:
def split_features(features):
    metros_cuadrados = np.nan
    ambientes = np.nan
    dormitorios = np.nan
    baños = np.nan
    cocheras = np.nan

    for feature in features:
        if 'm²' in feature:
            metros_cuadrados = int(feature.split()[0])
        elif 'amb.' in feature:
            ambientes = int(feature.split()[0])
        elif 'dorm.' in feature:
            dormitorios = int(feature.split()[0])
        elif 'baños' in feature:
            baños = int(feature.split()[0])
        elif 'coch.' in feature:
            cocheras = int(feature.split()[0])

    return pd.Series([metros_cuadrados, ambientes, dormitorios, baños, cocheras])

df[['metros_cuadrados', 'ambientes', 'dormitorios', 'baños', 'cocheras']] = df['Features'].apply(split_features)
df = df.drop("Features", axis=1)

In [128]:
df.value_counts("currency")

currency
USD          9570
ARS          8126
Consultar     342
nan             8
Name: count, dtype: int64

In [129]:
df_ars = df.copy()
df_ars = df_ars[df_ars["currency"] == "ARS"].reset_index(drop = True)
df_ars['Price'] = df_ars['Price'].str.replace(r'[^\d\.]', '', regex=True)
df_ars['Price'] = df_ars['Price'].str.replace('.', '')
df_ars['Price'] = df_ars['Price'].astype(float)
df_ars['Expensas'] = df_ars['Expensas'].str.replace(r'[^\d\.]', '', regex=True)
df_ars['Expensas'] = df_ars['Expensas'].str.replace('.', '')
df_ars['Expensas'] = df_ars['Expensas'].astype(float)
df_ars["Expensas / alquiler"] = df_ars["Expensas"] / df_ars["Price"]
df_ars["Expensas / m2"] = df_ars["Expensas"] / df_ars["metros_cuadrados"]

In [130]:
df_usd = df.copy()
df_usd = df_usd[df_usd["currency"] == "USD"].reset_index(drop = True)
df_usd['Price'] = df_usd['Price'].str.replace(r'[^\d\.]', '', regex=True)
df_usd['Price'] = df_usd['Price'].str.replace('.', '')
df_usd['Price'] = df_usd['Price'].astype(float)
df_usd['Expensas'] = df_usd['Expensas'].str.replace(r'[^\d\.]', '', regex=True)
df_usd['Expensas'] = df_usd['Expensas'].str.replace('.', '')
df_usd['Expensas'] = df_usd['Expensas'].astype(float)

In [131]:
df_ars["Expensas"].isna().sum()

2405

In [132]:
df_ars

,Price,Location,Address,Description,Expensas,Link,currency,metros_cuadrados,ambientes,dormitorios,baños,cocheras,Expensas / alquiler,Expensas / m2
0,260000.0,"Flores, Capital Federal",NaN,Dos ambientes dividido con Durlock. Oficinas y...,32760.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,53.0,2.0,NaN,NaN,NaN,0.126000,618.113208
1,650000.0,"Núñez, Capital Federal",NaN,Ph de 3 ambientes con patio- Ubicado en la esq...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,96.0,3.0,2.0,2.0,NaN,NaN,NaN
2,450000.0,"Caballito, Capital Federal",NaN,Semipiso departamento de tres ambientes al fre...,45000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,60.0,3.0,2.0,NaN,NaN,0.100000,750.000000
3,138000.0,"Centro / Microcentro, Capital Federal",NaN,Xintel(lor-lor-1690) Alquiler de Local en Cent...,52000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,59.0,NaN,NaN,NaN,NaN,0.376812,881.355932
4,800000.0,"Parque Patricios, Capital Federal",NaN,Excelente galpon en la mejor zona de parque pa...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,280.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8121,700000.0,"Recoleta, Capital Federal",NaN,"Excelente tres ambientes Recoleta, un departam...",85000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,70.0,3.0,2.0,NaN,NaN,0.121429,1214.285714
8122,150000.0,"Liniers, Capital Federal",NaN,Xintel(fde-fde-930) Alquiler de Local en Linie...,25000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,30.0,NaN,NaN,NaN,NaN,0.166667,833.333333
8123,500000.0,"Villa Devoto, Capital Federal",NaN,Local alquiler Villa Devoto. Amplio local en a...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,70.0,NaN,NaN,NaN,NaN,NaN,NaN
8124,250000.0,"Villa Crespo, Capital Federal",NaN,Local comercial sobre av. Scalabrini ortiz N° ...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,70.0,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
df_ars = df_ars[
                (df_ars["Expensas"]<=600_000)
                & (df_ars["Expensas"].notna())
                & (df_ars["Expensas"]>=5_000)
                & (df_ars["Price"]<=600_000)
                & (df_ars["Price"]>=10_000)
                & (df_ars["Expensas / alquiler"]<=1)
                & (df_ars["Expensas / m2"]<=10_000)
                ]
df_ars = df_ars.sort_values("Location")

In [134]:
df_ars.sort_values("Expensas / m2").iloc[0,5]

'https://www.zonaprop.com.ar/propiedades/clasificado/alclapin-muy-lindo-3-ambientes-chicos-52751162.html'

In [135]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Expensas / m2"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})
fig.update_xaxes(type = "category")

In [136]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Price"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [137]:
tmp_df = df_ars.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Expensas / m2"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250})

In [138]:
df_usd = df_usd[
    (df_usd["metros_cuadrados"]<=60)
    & (df_usd["metros_cuadrados"].notna())
    & (df_usd["Price"] <= 2_000)
]

In [139]:
df_usd.sort_values("Price", ascending=True)[:20].iloc[18,5]

'https://www.zonaprop.com.ar/propiedades/clasificado/alclocin-oficina-microcentro-calle-florida-52835451.html'

In [140]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_usd["Location"],y = df_usd["Price"], mode = "markers"))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [141]:
tmp_df = df_usd.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Price"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  )

In [142]:
fig = go.Figure()
fig.add_trace(go.Box(y = df_usd["Price"], x = df_usd["Location"]))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  height = 1_000)
fig.show()